In [1]:
# importing libraries for data modeling

import pandas as pd
import numpy as np
import collections
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer
from common import utils, vocabulary
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import preprocessing
import tensorflow as tf
import os
import time
import datetime
from CNN.text_cnn import TextCNN
from tensorflow.contrib import learn
import sys

/home/alexander_mpa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# loading model data 

normal_train = pd.read_csv("data/wiki_matched_short.train.0", delimiter="\t", header = None, names = ['sentence_text'])
simple_train = pd.read_csv("data/wiki_matched_short.train.1", delimiter="\t", header = None, names = ['sentence_text'])
normal_test = pd.read_csv("data/wiki_matched_short.test.0", delimiter="\t", header = None, names = ['sentence_text'])
simple_test = pd.read_csv("data/wiki_matched_short.test.0", delimiter="\t", header = None, names = ['sentence_text'])


normal_simple_style_test = pd.DataFrame(data = open("data/wiki_matched.test.0.tsf.normal-simple").readlines(), columns = ['sentence_text'])

simple_normal_style_test = pd.DataFrame(data = open("data/wiki_matched.test.1.tsf.simple-normal").readlines(), columns = ['sentence_text'])

normal_train['party']='normal'
normal_test['party']='normal'
simple_train['party']='simple'
simple_test['party']='simple'

normal_simple_style_test['party']='simple'
simple_normal_style_test['party']='normal'

#combine data together for training/testing
#shuffle it for random order
all_train = normal_train.append(simple_train)
all_test = normal_test.append(simple_test)

all_style_test = normal_simple_style_test.append(simple_normal_style_test)

all_train = all_train.sample(frac=.25)
all_test = all_test.sample(frac=.1)

all_style_test = all_style_test.sample(frac=.1)

print(all_train.head())
print(all_test.head())
print(all_style_test.head())

                                            sentence_text   party
61495   this was an early description of a process of ...  simple
167589  the color at right is called puce in the NP co...  normal
134427  even if NP had not followed strictly tradition...  normal
103428      NP NP NP ( NP ) is a large university in NP .  simple
149453  the victory paved the way for NP ' s forces to...  normal
                                         sentence_text   party
231  forty-nine species of pipefish and nine specie...  simple
201  NP NP had its own postal administration , sepa...  normal
13   today NP is organised as an independent , priv...  normal
266  out of DGDGDGDGDG participants in the national...  normal
48   it was discovered by NP NP . NP in images from...  normal
                                         sentence_text   party
115  it is the second of the storm of the NP , whic...  simple
349                ` ` <unk> ' ' and ` ` <unk> ' ' .\n  simple
2    the <unk> can be used to be the 

In [3]:
train_x = all_train['sentence_text']
train_y = all_train['party']
test_x = all_test['sentence_text']
test_y = all_test['party']

style_test_x = all_style_test['sentence_text']
style_test_y = all_style_test['party']

In [4]:
# tokenize data in comment body

tokenizer = TweetTokenizer()

train_x_tokens = []
test_x_tokens = []
style_test_x_tokens = []

for x in train_x:
    train_x_tokens.append(tokenizer.tokenize(x))
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))
    
for x in style_test_x:
    style_test_x_tokens.append(tokenizer.tokenize(x))

In [5]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=20000)
print("Vocabulary size: {:,}".format(vocab.size))
print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []
style_test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))
    
for x in style_test_x_tokens:
    style_test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 20,000
Vocabulary dict:  {'<s>': 0, '</s>': 1, '<unk>': 2, 'NP': 3, 'the': 4, '.': 5, ',': 6, 'of': 7, 'in': 8, 'and': 9, 'a': 10, 'to': 11, "'": 12, 'is': 13, 'DGDGDGDG': 14, 'was': 15, '`': 16, '(': 17, ')': 18, 'on': 19, 'as': 20, 'for': 21, 'by': 22, 'he': 23, 'it': 24, 'DGDG': 25, 'are': 26, 'with': 27, 's': 28, 'that': 29, 'his': 30, 'from': 31, 'at': 32, 'an': 33, 'DG': 34, 'or': 35, 'also': 36, 'this': 37, 'were': 38, 'be': 39, 'has': 40, 'which': 41, 'first': 42, 'have': 43, 'they': 44, 'one': 45, 'not': 46, 'had': 47, 'their': 48, 'its': 49, 'after': 50, ':': 51, 'other': 52, 'she': 53, 'can': 54, 'but': 55, 'her': 56, 'most': 57, 'two': 58, 'there': 59, 'used': 60, 'DGDGDG': 61, 'when': 62, 'many': 63, '-': 64, 'called': 65, 'been': 66, 'some': 67, 'who': 68, 'made': 69, 'more': 70, 'into': 71, 'all': 72, 'known': 73, 'became': 74, 'people': 75, 'time': 76, 'born': 77, 'about': 78, 'only': 79, 'during': 80, 'years': 81, 'these': 82, 'later': 83, 'than': 84, 

In [6]:
# count token occurences and convert to feature vector for BoW model

train_x_fdict = []
test_x_fdict = []
style_test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))
    
for x in style_test_x_ids:
    # remove democrat/republican tag from style test data
    del x[0]
    style_test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []
style_test_x_vector = []

num_features = vocab.size
print('creating id lists...')
for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in style_test_x_fdict:
    style_test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)
style_test_x_sparse_bow = utils.id_lists_to_sparse_bow(style_test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model
print('training Multinomial Naive Bayes for simple baseline model...')
nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)
y_pred_style = nb.predict(style_test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

acc_style = accuracy_score(style_test_y, y_pred_style)
print("Accuracy on style test set: {:.02%}".format(acc_style))
classrep_style = classification_report(style_test_y, y_pred_style)
print(classrep_style)

creating id lists...
starting id lists to sparse bow conversion...
training Multinomial Naive Bayes for simple baseline model...
Accuracy on test set: 63.64%
             precision    recall  f1-score   support

     normal       0.61      0.79      0.69        28
     simple       0.68      0.48      0.57        27

avg / total       0.65      0.64      0.63        55

Accuracy on style test set: 47.22%
             precision    recall  f1-score   support

     normal       0.41      0.33      0.37        33
     simple       0.51      0.59      0.55        39

avg / total       0.46      0.47      0.46        72



In [8]:
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids
style_test_x_neural = style_test_x_ids

for x in train_x_neural:
    # remove democrat/republican tag from train data
    del x[0]
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in style_test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model --update your class lables below!!

terms_list = [['normal'],['simple']]
lb = preprocessing.MultiLabelBinarizer()
lb.fit(terms_list)

train_labels = np.array(train_y.tolist())
test_labels = np.array(test_y.tolist())
style_test_labels = np.array(style_test_y.tolist())

train_lables_format = [train_labels[i:i+1] for i in range(0, len(train_labels), 1)]    
test_lables_format = [test_labels[i:i+1] for i in range(0, len(test_labels), 1)]
style_test_lables_format = [style_test_labels[i:i+1] for i in range(0, len(style_test_labels), 1)]

train_y_neural = lb.transform(train_lables_format)
test_y_neural = lb.transform(test_lables_format)
style_test_y_neural = lb.transform(test_lables_format)

In [ ]:
# creating and training CNN for deep classification
# CNN model structure and code based on an implementation of Kim Yoon's "Convolutional Neural Networks for Sentence Classification"
# From https://github.com/dennybritz/cnn-text-classification-tf

# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1,2,3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 50, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7fe3489f96d8>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7fe3489f9320>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7fe3489f9588>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7fe3489f92e8>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7fe3001aba58>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7fe3489f94e0>
F=<absl.flags._flag.Flag object at 0x7fe2633d5198>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7fe348a76f60>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7fe3489f9278>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7fe3489f97b8>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7fe3489f9438>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7fe3489f9080>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7fe348a76f28>

x_train (94368, 265)
y_train (94368, 2)
x_dev (55, 265)
x_dev (55, 2)
Vocabulary Size: 20000
Train/Dev split: 94368/55
INFO:tensorflow:Summary name em

In [ ]:
# running CNN model against style transferred test data

# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

# Eval Parameters
# Update these checkpoint locations of the latest model from the log output of the cell above
tf.flags.DEFINE_string("checkpoint_dir", "/home/alexander_mpa/capstone/Parlancr/models/classifier/runs/1533580012/checkpoints/", "Checkpoint directory from training run")
checkpoint_file = "/home/alexander_mpa/capstone/Parlancr/models/classifier/runs/1533580012/checkpoints/model-6700"

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

x_test = np.array(style_test_x_neural)
y_test = np.array(style_test_y_neural)


print("\nEvaluating...\n")

# Evaluation
# ==================================================
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy

lb = preprocessing.LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
lb.fit(np.asarray(style_test_y.tolist()).flatten())  
style_test_y_neural_acc = lb.transform(np.asarray(style_test_y.tolist()))

acc_style_neural = accuracy_score(style_test_y_neural_acc, all_predictions)
print("Accuracy on style test set: {:.02%}".format(acc_style_neural))
classrep_style_neural = classification_report(style_test_y_neural_acc, all_predictions)
print(classrep_style)